# 🧠 Mental Health in Students Post-COVID: A Silent Crisis?
#### 📊 Preprocessing Notebook + Exploratory Data Analysis (EDA)


> *"The classrooms reopened, but something never quite returned."*  
> *"Smiles came back, but the silence grew louder."*

The COVID-19 pandemic reshaped the world in ways we are still trying to understand—and among its most silent victims were **students**. Behind the return to academic routine lies a shadow crisis: **increased anxiety, depression, sleep disorders, and even suicidal thoughts**. This project aims to explore and analyze the **mental health impact on students post-COVID**, using real-world data that includes psychological, academic, lifestyle, and demographic variables.

Through this notebook, we will:
- 🧹 Perform **data preprocessing** to clean and shape the dataset.
- 📊 Conduct **exploratory data analysis (EDA)** to uncover patterns, correlations, and warning signs.
- 🎯 Use visualizations that not only highlight trends, but **tell the story of students' struggles** in a post-pandemic world.

This is not just a statistical exercise — it's an attempt to quantify a **crisis hidden in plain sight**.

Let’s begin.

## 🔧 Imports
Below are the necessary libraries required for data handling, visualization, and basic preprocessing.


In [7]:
# 📦 Basic Imports
import numpy as np
import pandas as pd

# 📊 Visualization Libraries
import matplotlib.pyplot as plt
import seaborn as sns

# 🛠️ Display Settings
plt.style.use('ggplot')
sns.set_palette('Set2')
import warnings
warnings.filterwarnings('ignore')

# 🧠 Optional: For interactive visuals (if you plan to add later)
# import plotly.express as px
# import cufflinks as cf
# cf.go_offline()

# 🧹 For Preprocessing
from sklearn.preprocessing import LabelEncoder


## 📂 Loading and Combining Datasets

We are working with **two publicly available datasets** focused on student mental health after pandemic.  
Both datasets have nearly identical structures, but we ensured consistency by applying minor modifications before merging them into a single dataframe.

> Final merged dataset will be the foundation of all our preprocessing and analysis.


In [85]:
# Load the first cleaned dataset (already preprocessed and saved)
df1 = pd.read_excel(r'datasets/final_depression_dataset_2.xlsx')

# Load the second dataset which had original raw data
df2 = pd.read_excel(r'datasets/student_depression_dataset.xlsx')

Modifications of both datasets before merging

In [86]:
#Drop unnecessary columns from both datasets
# These columns are not needed for our analysis and visualization
df1 = df1.drop(columns=['Profession', 'Work Pressure', 'Job Satisfaction'])
df2 = df2.drop(columns=['Profession', 'Work Pressure', 'Job Satisfaction'])

In [87]:
#Renaming columns for consistency
# This ensures both datasets have the same column names for merging
df1 = df1.rename(columns={'Have you ever had suicidal thoughts ?': 'Suicidal Thoughts',
                        'Work/Study Hours': 'Study Hours',})
df2 = df2.rename(columns={'Have you ever had suicidal thoughts ?': 'Suicidal Thoughts',
                        'Work/Study Hours': 'Study Hours',})

Merge both DataFrames

In [88]:
#Merged both DataFrames using pd.concat
# This combines the two datasets into a single DataFrame for analysis
merged_df = pd.concat([df1, df2], ignore_index=True)

You can view your new Dataframe information 

In [89]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28403 entries, 0 to 28402
Data columns (total 14 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Gender                            28403 non-null  object 
 1   Age                               28403 non-null  int64  
 2   City                              28403 non-null  object 
 3   Academic Pressure                 28403 non-null  int64  
 4   CGPA                              28403 non-null  float64
 5   Study Satisfaction                28403 non-null  int64  
 6   Sleep Duration                    28403 non-null  object 
 7   Dietary Habits                    28403 non-null  object 
 8   Degree                            28403 non-null  object 
 9   Suicidal Thoughts                 28403 non-null  object 
 10  Study Hours                       28403 non-null  int64  
 11  Financial Stress                  28403 non-null  object 
 12  Fami

We will see how our new DataFrame is looking 

In [90]:
merged_df.head(5)

,Gender,Age,City,Academic Pressure,CGPA,Study Satisfaction,Sleep Duration,Dietary Habits,Degree,Suicidal Thoughts,Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,Male,28,Varanasi,2,6.51,4,7-8 hours,Moderate,BA,Yes,9,2,Yes,No
1,Male,28,Bangalore,4,7.48,5,5-6 hours,Healthy,MD,Yes,7,1,Yes,No
2,Male,25,Rajkot,1,7.21,3,5-6 hours,Unhealthy,MBBS,Yes,10,4,No,Yes
3,Male,23,Mumbai,1,9.90,4,More than 8 hours,Unhealthy,MSc,Yes,7,2,Yes,No
4,Female,31,Vadodara,1,5.97,5,More than 8 hours,Healthy,LLM,Yes,4,2,Yes,No


#### 🧹 Data Preprocessing & Normalization

Before we dive into exploratory analysis, we will perform essential preprocessing steps and normalize categorical column values to ensure consistency and enhance the clarity of our visualizations.


Convert Male/Female to M/F

In [91]:
#Using map function to convert Male/Female to M/F
# This creates cleaner labels for plots and improves readability in visual analysis
merged_df['Gender'] = merged_df['Gender'].map({'Male' : 'M' , 'Female':'F'})

#### Convert All columns who has Yes/No to 1/0
Converting to 1/0 simplifies analysis and makes the data ready for statistical and machine learning models.

In [92]:
#We will create a list of columns that have Yes/No values
# Converting these columns to 1/0 for easier analysis
lst = ['Suicidal Thoughts' , 'Family History of Mental Illness' , 'Depression']

#Using loop to iterate through the list and map Yes/No to 1/0
# This simplifies the data for statistical analysis and visualization
for columns in lst:
    merged_df[columns] = merged_df[columns].map({'Yes': '1', 'No': '0'})

In [93]:
merged_df['Sleep Duration'].value_counts()
#You can see here due to comma we have repeated values in different formats
#We will remove those commas to make it consistent
#Then we will convert the column to numeric type for better analysis

Sleep Duration
'Less than 5 hours'    8310
'7-8 hours'            7346
'5-6 hours'            6183
'More than 8 hours'    6044
7-8 hours               128
More than 8 hours       128
5-6 hours               123
Less than 5 hours       123
Others                   18
Name: count, dtype: int64

Convert Sleep Duration into numeric

In [94]:
#we will create a mapping dictionary to convert sleep duration strings to numeric values
mapping = {
    '5-6 hours': 5.5,
    "'5-6 hours'": 5.5,
    '7-8 hours': 7.5,
    "'7-8 hours'": 7.5,
    'More than 8 hours': 9,
    "'More than 8 hours'": 9,
    'Less than 5 hours': 4.5,
    "'Less than 5 hours'": 4.5,
    'Others': np.nan  # You can impute or drop later
}

#By using the map function and mapping dict, we can convert the 'Sleep Duration' column to numeric values
merged_df['Sleep Duration'] = merged_df['Sleep Duration'].map(mapping)

We have 18 Null Values due to Other 

In [95]:
merged_df['Sleep Duration'].isnull().sum()

18

In [96]:
#Using fillna to handle missing values in 'Sleep Duration'
# This replaces NaN values with the mean of the column for better analysis
merged_df['Sleep Duration'].fillna(merged_df['Sleep Duration'].mean(), inplace=True)

In [97]:
#Now you can check the number of null values in 'Sleep Duration' again
merged_df['Sleep Duration'].isnull().sum()

0

Convert Healthy, Moderate and Unhealthy into Numeric values for better analysis

In [98]:
#Convert Healthy, Moderate and Unhealthy into Numeric values for better analysis
# This will help in statistical analysis and visualization
merged_df['Dietary Habits'] = merged_df['Dietary Habits'].map({'Healthy':'2' , 'Unhealthy':'0' , 'Moderate':'1'})

Delete Rows who's Age is Greater than 27 for focusing on Student Age

In [105]:
merged_df = merged_df[merged_df['Age'] < 28]

Removing Invalid City Rows

In [ ]:
#When you see there are some rows who doesn't have a valid city name
# We will remove those rows to focus on valid city data
merged_df['City'].value_counts()

In [ ]:

# Remove rows where the city count is less than 20
city_counts = merged_df['City'].value_counts()
#This will filter out cities that have less than 20 entries
valid_cities = city_counts[city_counts >= 20].index
# Filter the DataFrame to keep only rows with valid cities
merged_df = merged_df[merged_df['City'].isin(valid_cities)]


Save Cleaned DataFrame into a Excel File

In [123]:
merged_df.to_excel(r'post_covid_mental_health_cleaned.xlsx', index=False)

In [125]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16838 entries, 2 to 28402
Data columns (total 14 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Gender                            16838 non-null  object 
 1   Age                               16838 non-null  int64  
 2   City                              16838 non-null  object 
 3   Academic Pressure                 16838 non-null  int64  
 4   CGPA                              16838 non-null  float64
 5   Study Satisfaction                16838 non-null  int64  
 6   Sleep Duration                    16838 non-null  float64
 7   Dietary Habits                    16838 non-null  int32  
 8   Degree                            16838 non-null  object 
 9   Suicidal Thoughts                 16838 non-null  int32  
 10  Study Hours                       16838 non-null  int64  
 11  Financial Stress                  16838 non-null  int32  
 12  Family Hi